# Identifying and anonymizing Personally Identifiable Information (PII) with ydata-sdk

Many datasets contain Personal Information (PI) and therefore cannot easily be shared. It is not enough to simply synthesize a dataset to remove the personal information. Indeed, although the synthetic dataset does not contain any records from the original data, it may still include values that represent personally identifiable information (PII). For instance, in some context, the simple fact to have the name of a city might leak information about the entire dataset.

To solve this problem, YData offers the possibility to anonymize any field such that the synthetic data do not contain any PI. The anonymizer mechanism provides several pre-configured anonymizer that corresponds to the most common scenarios (city, address, names, IP address) and also allows to specify a regular expression to match any format that you might have (e.g. an internal customer ID format).

By the end of this notebook, you will learn how to:
- Detect columns that contain PII
- Apply built-in anonymization strategies (e.g., names, locations, IDs)
- Generate a sanitized version of your dataset using `ydata-sdk`

The dataset used in this notebook can be found at https://www.kaggle.com/datasets/yeanzc/telco-customer-churn-ibm-dataset

## Authenticate with your YData account

In [1]:
# Authenticate with your ydata-sdk token - https://dashboard.ydata.ai/
import os

os.environ['YDATA_LICENSE_KEY'] = '{add-your-key}'

## Indentifying PII with ydata-sdk

In this section, we simply load the dataset and display few rows to observe that there are personal information that should be anonymized.

In [2]:
import pandas as pd

from ydata.dataset import Dataset

# Step 1: Load or create your Dataset
df = pd.read_csv('insert-file-path')
dataset = Dataset(df)

In [3]:
df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,LP001015,Male,Yes,0,Graduate,No,5720,0,110.0,360.0,1.0,Urban
1,LP001022,Male,Yes,1,Graduate,No,3076,1500,126.0,360.0,1.0,Urban
2,LP001031,Male,Yes,2,Graduate,No,5000,1800,208.0,360.0,1.0,Urban
3,LP001035,Male,Yes,2,Graduate,No,2340,2546,100.0,360.0,NaN,Urban
4,LP001051,Male,No,0,Not Graduate,No,3276,0,78.0,360.0,1.0,Urban


In [6]:
from ydata.metadata import Metadata

# Step 2: Calculate the metadata
# When infer_characteristics is set to True, ydata-sdk will automatically infer data characteristics such as potential PII (e.g., email, name, etc.). By default, this option is set to False.
metadata = Metadata(dataset, infer_characteristics = True)
metadata

[########################################] | 100% Completed | 105.24 ms
[########################################] | 100% Completed | 105.03 ms
[########################################] | 100% Completed | 103.13 ms


In [7]:
metadata.summary['characteristics']

{'Loan_ID': [{'characteristic': <ColumnCharacteristic.ID: 'id'>,
   'value': 1,
   'upper_bound': 1,
   'lower_bound': 1}]}

You can also define characteristics manually using the `characteristics` parameter.

This parameter expects a dictionary that explicitly maps column names to known PII types. This is useful when you already know which columns contain sensitive information. The format should be:

{ "column_name": "pii_type" }

For example:
{ "email": "email", "customer_id": "id" }

## Anonymizing PII with ydata-sdk

Now that we were able to identify potential PII columns, it is also possible to create the logic to anonymize the columns Now that we've identified potential PII columns, we can define the logic to anonymize them using the Fabric Anonymizer.

In this example, we'll demonstrate how to anonymize two specific columns: customerID and City.

**Note:** This example focuses on demonstrating the anonymization workflow — not on fully securing the dataset. Other columns such as Lat Long, Latitude, Longitude, Zip Code, State, and Country may also contain sensitive information and should be reviewed carefully in a real-world scenario.

To view the list of available built-in anonymization methods, use the following command:

In [8]:
from ydata.preprocessors.methods.anonymization import AnonymizerType

## Available anonymizer types list
dict(AnonymizerType.__members__)

{'REGEX': <AnonymizerType.REGEX: 1>,
 'IP': <AnonymizerType.IP: 2>,
 'IPV4': <AnonymizerType.IPV4: 3>,
 'IPV6': <AnonymizerType.IPV6: 4>,
 'HOSTNAME': <AnonymizerType.HOSTNAME: 5>,
 'LICENSE_PLATE': <AnonymizerType.LICENSE_PLATE: 6>,
 'ABA': <AnonymizerType.ABA: 7>,
 'BANK_COUNTRY': <AnonymizerType.BANK_COUNTRY: 8>,
 'BBAN': <AnonymizerType.BBAN: 9>,
 'IBAN': <AnonymizerType.IBAN: 10>,
 'SWIFT': <AnonymizerType.SWIFT: 11>,
 'BARCODE': <AnonymizerType.BARCODE: 12>,
 'COMPANY': <AnonymizerType.COMPANY: 13>,
 'COMPANY_SUFFIX': <AnonymizerType.COMPANY_SUFFIX: 14>,
 'COMPANY_EMAIL': <AnonymizerType.COMPANY_EMAIL: 15>,
 'EMAIL': <AnonymizerType.EMAIL: 16>,
 'DOMAIN_NAME': <AnonymizerType.DOMAIN_NAME: 17>,
 'MAC_ADDRESS': <AnonymizerType.MAC_ADDRESS: 18>,
 'PORT_NUMBER': <AnonymizerType.PORT_NUMBER: 19>,
 'URI': <AnonymizerType.URI: 20>,
 'USER_NAME': <AnonymizerType.USER_NAME: 21>,
 'JOB': <AnonymizerType.JOB: 22>,
 'FIRST_NAME': <AnonymizerType.FIRST_NAME: 23>,
 'FIRST_NAME_FEMALE': <Anonym

For `CustomerID` anonymization we will use a REGEX as the value are specific to this dataset.    
On the other hand, `City` can leverage the `AnonymizerType.CITY` to generate fake city names.

The configuration is to be passed to the Synthesizer model and looks like the following mapping:

In [9]:
# Step 1: Configure the Anonymizer – Define which masking or replacement strategy should be applied to each PII column based on its type and sensitivity.
anonymizer_config = {
    'Loan_ID': {'type': 'regex', 'regex': r'[0-9]{4}-[A-Z]{5}'},  # Regex as a string is deduced automatically as AnonymizerType.REGEX
    #'City': AnonymizerType.CITY  # Direct usage of AnonymizerType
}

In [10]:
from ydata.preprocessors.preprocess_methods import AnonymizerEngine

# Step 2: Create your Anonymizer Engine
anonymizer = AnonymizerEngine()
dataset_anonymized = anonymizer.fit_transform(X=dataset, 
                                      config=anonymizer_config, 
                                      metadata=metadata)

dataset_anonymized.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,1080-EPXMZ,Male,Yes,0,Graduate,No,5720,0,110.0,360.0,1.0,Urban
1,2038-EMCSV,Male,Yes,1,Graduate,No,3076,1500,126.0,360.0,1.0,Urban
2,7987-VIWFX,Male,Yes,2,Graduate,No,5000,1800,208.0,360.0,1.0,Urban
3,6188-XZFFB,Male,Yes,2,Graduate,No,2340,2546,100.0,360.0,NaN,Urban
4,6670-JTLDE,Male,No,0,Not Graduate,No,3276,0,78.0,360.0,1.0,Urban
